In [ ]:
!pip install insightface 'onnxruntime-gpu<1.22.0' gdown

In [1]:
from pathlib import Path
import shutil
from torch.utils.data import DataLoader, Subset
import requests
import gdown
import tarfile

ROOT = Path("./data")
if not ROOT.exists():
    ROOT.mkdir()
    
DATASET_PATH = ROOT / "casia_webface"
if not DATASET_PATH.exists():
    tmp = ROOT / "tmp.tar.gz" 
    gdown.download(id="199O-Ud7FLOjTztVruQ3pjAPLX_AoPj16", output=str(tmp))
    with tarfile.TarFile(str(tmp)) as tar_ref:
        tar_ref.extractall(str(ROOT))
    tmp.unlink()
    shutil.move(str(ROOT / "casia_webface"), str(DATASET_PATH))

In [ ]:
from threading import Lock
from insightface.app import FaceAnalysis
import torch
import cv2
from concurrent.futures import ThreadPoolExecutor

CUDA_AVAILABLE = torch.cuda.is_available()
NUM_WORKERS = 4
LOCK = Lock()
CNT = 0

face_apps: list[FaceAnalysis] = []
for i in range(NUM_WORKERS):
    print(f"Preparing {i}")
    app = FaceAnalysis(name='buffalo_l', providers=[("CUDAExecutionProvider", {"device_id": 0})])  # Use 'CUDAExecutionProvider' for GPU
    app.prepare(ctx_id=(-1 if not CUDA_AVAILABLE else 0))  # ctx_id=-1 for CPU, 0 for GPU
    print(f"Prepared {i}")
    face_apps.append(app)

def run(img: str):
    global CNT
    with LOCK:
        app = face_apps[CNT]
        CNT = (CNT + 1) % NUM_WORKERS

    return app.get(cv2.imread(img))

pool = ThreadPoolExecutor(NUM_WORKERS)

Preparing 0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /home/erb/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionPr

In [ ]:
from torch import nn

class MetricsNet(nn.Module):
    def __init__(self, d: int):
        super().__init__()
        
        layers = []
        while d > 1:
            layers.extend([nn.Linear(d, d//2), nn.ReLU()])
            d = d // 2
        layers.extend([nn.Linear(2, 1)])
        
        self.seq = nn.Sequential(layers)
        
    def forward(self, x):
        return self.seq(x)


# 1. Definir Neural Network Metrics+
# 2. Treinar localmente...
# 3. Treinar no kaggle, salvar checkpoints.
# 4. treinar ista-net com base nisso...
# 5. 

In [ ]:
from typing import Generator, NoReturn
from torchvision.datasets import ImageFolder
import cv2
import torch
from random import Random

def get_class_to_idx_list(dataset: ImageFolder) -> list[list[int]]:
    ret = []
    cur = []
    last = dataset.targets[0]
    for i, s in enumerate(dataset.targets):
        if s != last:
            last = s
            ret.append(cur)
            cur = []
        cur.append(i)
    ret.append(cur)
    return ret


def generate_batches(dataset: ImageFolder, generator: Random, batch_size: int) -> Generator[list, any, NoReturn]:
    class_to_idx_list = get_class_to_idx_list(dataset)
    classes = set(range(len(class_to_idx_list)))
    while True:
        batch = []
        
        if (mod := batch_size % 6) != 0:
            batch_size = batch_size + 6 - mod

        for i in range(batch_size//6):
            cur_class = generator.choice(classes)
            
            for _ in range(3):
                same_img1_idx, same_img2_idx = generator.choices(class_to_idx_list[cur_class], k=2)
                batch.append((dataset[same_img1_idx], dataset[same_img2_idx], True))

            for _ in range(3):
                diff_img_class1 = generator.choice(classes - {cur_class})
                diff_img1_idx = generator.choice(class_to_idx_list[diff_img_class1])
                diff_img2_idx = generator.choice(class_to_idx_list[cur_class])
                batch.append((dataset[diff_img1_idx], dataset[diff_img2_idx], False))

        yield batch

dataset = ImageFolder(DATASET_PATH, loader=lambda pth: torch.from_numpy(cv2.imread(str(pth))))
generator = Random("abcd")

# 1. train()
# 2. test()
# 3. loss func()
# 4. loop de treino().
#
# 5. u-net.